# **SCORING METHODE FOR TREND DATA**

In [ ]:
# Importation du package pandas
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.lines as mlines
import matplotlib.transforms as mtransforms
import matplotlib.pyplot as plt
%matplotlib inline

# Dataframe

In [ ]:
df = pd.read_csv('trend2022.csv', header=0, sep=';', encoding='latin-1')
df

In [ ]:
for numero, nom_colonne in enumerate(df.columns):
    print(numero,":", nom_colonne)

In [ ]:
trend = df.drop(df.columns[1:21], axis = 1)
trend

# Introduct new indiv

In [ ]:
new_individual = pd.DataFrame([1] * trend.shape[1], index=trend.columns).T

# Ajouter cette ligne à la fin du DataFrame
trend = pd.concat([trend, new_individual], ignore_index=True)

In [ ]:
new_individual = pd.DataFrame([0] * trend.shape[1], index=trend.columns).T

# Ajouter cette ligne à la fin du DataFrame
trend = pd.concat([trend, new_individual], ignore_index=True)

In [ ]:
trend

In [ ]:
trend.dtypes

In [ ]:
trend.isna().sum()

In [ ]:
trend.describe()

In [ ]:
summary = trend.describe()

var = summary.loc['mean'][summary.loc['mean'] > 0.4]

print("Variables avec une moyenne supérieure à 0.4 :")
for variable, mean in var.items():
    print(f"{variable}: {mean}")

# Score

## Score sur la Moyenne des domaines

In [ ]:
prefixes = [f'X{i}' for i in range(1, 16)]
new_df = pd.DataFrame()

# Pour chaque préfixe, la somme des colonnes correspondantes est:
for prefix in prefixes:
    cols_to_sum = [col for col in df.columns if col.startswith(prefix)]
    new_df[prefix] = trend[cols_to_sum].sum(axis=1)

new_df

In [ ]:
break

In [ ]:
v1 = new_df.copy()
# v1.to_csv('norm_numb_policy.csv', index=False)

In [ ]:
new_df.describe()

In [ ]:
for prefix in prefixes:
    bad = min(new_df[prefix])
    good = max(new_df[prefix])
    new_df[f'{prefix}m'] = ((new_df[prefix] - bad) / (good - bad)) * (10 - 1) + 1

In [ ]:
n_col = [f'{prefix}m' for prefix in prefixes]
new_df['note'] = np.sqrt(np.mean(np.square(new_df[n_col]), axis=1))
new_df['note'] = round(new_df['note'])
new_df['note'] = new_df['note'].astype(int)

In [ ]:
new_df

In [ ]:
v2 = new_df.iloc[:, 15:31]
v2

In [ ]:
break

In [ ]:
v2 = v2.iloc[:-2]
# v2.to_csv('policy_index.csv', index=False)

In [ ]:
ndf = pd.concat([trend['Trade.Agreement'], new_df['note']], axis = 1)
ndf = ndf.iloc[:-2]
ndf

In [ ]:
import matplotlib.pyplot as plt

# Créer un histogramme pour visualiser la distribution
plt.hist(ndf['note'], bins=20, color='blue', edgecolor='black')

# Ajouter des labels et un titre
plt.xlabel('IAE')
plt.ylabel('Effectif')
plt.title('IAE')

# Afficher le plot
plt.show()

In [ ]:
# Table de distribution de la variable standardisée et mise à l'échelle
distribution_table = ndf['note'].value_counts().sort_index().reset_index()
distribution_table.columns = ['Valeur', 'Fréquence']

print(distribution_table)

## Score sur Sum Nb de clauses

In [ ]:
trend['Total'] = trend.iloc[:, 1:].sum(axis=1)
trend

In [ ]:
bad_valeur = min(trend['Total'])
good_valeur = max(trend['Total'])

# Appliquer la transformation Min-Max Scaling
trend['IAE_sum'] = round(((trend['Total'] - bad_valeur) / (good_valeur - bad_valeur)) * (10 - 1) + 1)
trend['IAE_sum'] = trend['IAE_sum'].astype(int)
trend = trend.iloc[:-2]

In [ ]:
import matplotlib.pyplot as plt

# Créer un histogramme pour visualiser la distribution
plt.hist(trend['IAE_sum'], bins=20, color='red', edgecolor='black')

# Ajouter des labels et un titre
plt.xlabel('IAE')
plt.ylabel('Effectif')
plt.title('IAE - Nombre total de normes')

# Afficher le plot
plt.show()

In [ ]:
# Table de distribution de la variable standardisée et mise à l'échelle
distribution_table = trend['IAE_sum'].value_counts().sort_index().reset_index()
distribution_table.columns = ['Valeur', 'Fréquence']

print(distribution_table)

## IRT

In [ ]:
T = trend.drop(trend.columns[[0, 299, 300]], axis = 1)
T

In [ ]:
new_ind1 = pd.DataFrame([1] * T.shape[1], index=T.columns).T
T = pd.concat([T, new_ind1], ignore_index=True)

new_ind0 = pd.DataFrame([0] * T.shape[1], index=T.columns).T
T = pd.concat([T, new_ind0], ignore_index=True)

T

In [ ]:
break

In [ ]:
from sklearn.decomposition import FactorAnalysis

In [ ]:
n_factors = 1  # Number of latent factors
fa = FactorAnalysis(n_components=n_factors)

# Fit the model to the data
fa.fit(T)

# Transform the data to get the latent factors
latent_factors = fa.transform(T)

# Get the factor score
factor_scores = fa.transform(T)

In [ ]:
T['factor_score'] = factor_scores
T

In [ ]:
bad_v = min(T['factor_score'])
good_v = max(T['factor_score'])

# Appliquer la transformation Min-Max Scaling
T['IAE_factor'] = round(((T['factor_score'] - bad_v) / (good_v - bad_v)) * (10 - 1) + 1)
T['IAE_factor'] = T['IAE_factor'].astype(int)
T = T.iloc[:-2]

In [ ]:
import matplotlib.pyplot as plt

# Créer un histogramme pour visualiser la distribution
plt.hist(T['IAE_factor'], bins=20, color='green', edgecolor='black')

# Ajouter des labels et un titre
plt.xlabel('IAE')
plt.ylabel('Effectif')
plt.title('IAE - Factor analysis')

# Afficher le plot
plt.show()

## Score Comparaison

In [ ]:
# Concaténer les colonnes sélectionnées de df1 avec df2
compare_sco = pd.concat([ndf['note'], trend['IAE_sum'], T['IAE_factor']], axis=1)
compare_sco.rename(columns={'note':'IAE'}, inplace=True)
compare_sco

In [ ]:
break

In [ ]:
# compare_sco.to_csv('compare_score.csv', index=False)

In [ ]:
compare_sco.describe()

# Data extraction

## Data by Trade Agreemnt

In [ ]:
df_slice = df.iloc[:, 3:21]
df_scores = pd.concat([df['Trade.Agreement'], df_slice, trend['Total'], ndf['note']], axis=1)
df_scores.rename(columns={'Total': 'Nombre de normes'}, inplace=True)
df_scores.rename(columns={'note': 'IAE'}, inplace=True)
df_scores

In [ ]:
break

In [ ]:
dx = pd.read_csv('infos_accords.csv')
dx = dx.drop(columns=['depth_index', 'mar_typedepth'])
df_scores = pd.merge(df_scores, dx, on = ['Trade.Agreement'], how = 'left')
df_scores

In [ ]:
# df_scores.to_csv("df_score.csv", index=False)
# print("Le fichier csv a été créé avec succès.")

## Data by country

In [ ]:
data_pays = pd.read_csv('trenddyadic2022.csv', header=0, sep=';', encoding='latin-1')
data_pays = data_pays.drop(data_pays.columns[24:322], axis = 1)
data_pays

In [ ]:
data_pays = pd.merge(data_pays, df_scores[['Trade.Agreement', 'Nombre de normes', 'IAE']], on='Trade.Agreement', how='left')
data_pays

In [ ]:
df1 = data_pays.drop(columns=['country2'])
df2 = data_pays.drop(columns=['country1'])


In [ ]:
df1.rename(columns={'country1': 'country'}, inplace=True)
df2.rename(columns={'country2': 'country'}, inplace=True)
data_pays = pd.concat([df1, df2], ignore_index=True)

In [ ]:
data_pays = data_pays.drop_duplicates(subset=['Trade.Agreement', 'country'])
data_pays

In [ ]:
break

In [ ]:
# data_pays.to_csv('data_accord_pays.csv', index=False)
# print("Le fichier csv a été créé avec succès.")

In [ ]:
country = data_pays.copy()
country

In [ ]:
country = country.drop(country.columns[0], axis = 1)
country = country.drop(country.columns[1:15], axis = 1)
country = country.drop(country.columns[2:8], axis = 1)
country

In [ ]:
break

In [ ]:
# country.to_excel('country_trend.xlsx', sheet_name = 'country score', index = False)
# print('Well successed !')

In [ ]:
break

**!!! Régler le problème des noms de pays avant de continuer en utlisant *'res_pays problems.xlsx'*, pour corriger dans *'country_trend.xlsx'* et en appelant la correction 'C'**

In [ ]:
country = pd.read_excel('country_trend.xlsx', sheet_name='country score')
country

In [ ]:
country['country'] = country['C']
country = country.drop(columns='C', axis = 1)
country.rename(columns={'Note':'IAE'}, inplace=True)

In [ ]:
country

In [ ]:
moy2 = country.groupby('country')['IAE'].mean().reset_index()
moy2 = moy2.rename(columns={'IAE': 'IAE moyen'})
moy2

In [ ]:
som = country.groupby('country')['Nombre de normes'].sum().reset_index()
som = som.rename(columns={'Nombre de normes': 'Nombre de normes signées'})
som['Nombre de normes signées'] = som['Nombre de normes signées'].astype(int)
som

In [ ]:
max = country.groupby('country')['Year'].max().reset_index()
max = max.rename(columns={'Year': 'YearLastPTASigned'})
max

In [ ]:
min = country.groupby('country')['Year'].min().reset_index()
min = min.rename(columns={'Year': 'YearFirstPTASigned'})
min

In [ ]:
country_trend = pd.concat([min, max['YearLastPTASigned'], som['Nombre de normes signées'], moy2['IAE moyen']], axis=1)
country_trend

In [ ]:
country_trend.describe()

In [ ]:
break

In [ ]:
# country_trend.to_csv('data_pays.csv', index = False)
# print('Well done !')

## Ajout de variables macro

In [ ]:
df_scores.drop(columns=['regioncon'], inplace=True)
df_scores

#### left jointure

In [ ]:
break

**!!! On corrige d'abord les noms des pays dans la base dyadic avant de les importer**

In [ ]:
data = pd.read_excel('dyadic_trend.xlsx', sheet_name='trenddyadic2022')
data.head()

In [ ]:
data['country1'] = data['C1']
data['country2'] = data['C2']

data = data.drop(columns = ['C1', 'C2'])
data

In [ ]:
df1 = data.drop(columns=['country2'])
df2 = data.drop(columns=['country1'])

In [ ]:
df1.rename(columns={'country1': 'country'}, inplace=True)
df2.rename(columns={'country2': 'country'}, inplace=True)
dat = pd.concat([df1, df2], ignore_index=True)

In [ ]:
dato = dat.drop_duplicates(subset=['Trade.Agreement', 'country'])
dato

#### all data

In [ ]:
POPTOT = pd.read_excel('all_DEP.xlsx', sheet_name='POPTOT')
EPI22 = pd.read_excel('all_DEP.xlsx', sheet_name='EPI2022')
EPI20 = pd.read_excel('all_DEP.xlsx', sheet_name='EPI2020')
CO2 = pd.read_excel('all_DEP.xlsx', sheet_name='CO2')

PIB = pd.read_excel('all_DEP.xlsx', sheet_name='GDP-PPP')
PCAP = pd.read_excel('all_DEP.xlsx', sheet_name='GDP-PPP-CAP')

EXP = pd.read_excel('all_DEP.xlsx', sheet_name='EXP')
MER = pd.read_excel('all_DEP.xlsx', sheet_name='MERCHEXP')

DEM22 = pd.read_excel('all_DEP.xlsx', sheet_name='DEM')
IDE = pd.read_excel('all_DEP.xlsx', sheet_name='INVEST')
VCC = pd.read_excel('all_DEP.xlsx', sheet_name='VCC')

In [ ]:
VCC

In [ ]:
POPTOT = POPTOT.drop(POPTOT.columns[[1, 2, 3]], axis = 1)
EPI20 = EPI20.drop(EPI20.columns[[0, 1, 2]], axis = 1)
CO2 = CO2.drop(CO2.columns[[0, 1]], axis = 1)

PIB = PIB.drop(PIB.columns[[0, 2]], axis = 1)
PCAP = PCAP.drop(PCAP.columns[[0, 2]], axis = 1)

EXP = EXP.drop(columns=['Indicator Name'])
MER = MER.drop(MER.columns[[0, 2, 3]], axis = 1)
IDE = IDE.drop(IDE.columns[[0, 1]], axis = 1)
VCC = VCC.drop(columns=['Country'])

In [ ]:
EPI20

In [ ]:
POPTOT = POPTOT.melt(id_vars='C', var_name='Year', value_name='pop')
PIB = PIB.melt(id_vars='C', var_name='Year', value_name='gdp')
PCAP = PCAP.melt(id_vars='C', var_name='Year', value_name='gdp_pcap')
EXP = EXP.melt(id_vars='C', var_name='Year', value_name='export')
MER = MER.melt(id_vars='C', var_name='Year', value_name='merch')

In [ ]:
PCAP

In [ ]:
EXP.dtypes

In [ ]:
list = [POPTOT, PIB, PCAP, EXP, MER, CO2, IDE]
for d in list:
    d['Year'] = d['Year'].astype(int)

In [ ]:
# Fusion des DataFrames
result = pd.merge(POPTOT, PIB, on=['C', 'Year'], how='left')
result = pd.merge(result, PCAP, on=['C', 'Year'], how='left')
result = pd.merge(result, EXP, on=['C', 'Year'], how='left')
result = pd.merge(result, MER, on=['C', 'Year'], how='left')
result = pd.merge(result, CO2, on=['C', 'Year'], how='left')
result = pd.merge(result, IDE, on=['C', 'Year'], how ='left')

In [ ]:
result = result.rename(columns = {'C':'country'})
result

In [ ]:
break

In [ ]:
# result.to_csv('all_DEP_consolidate.csv', index = False)
# print('well !')

In [ ]:
result.isna().sum()

In [ ]:
result.dtypes

In [ ]:
dato['Year'] = dato['Year'].astype(int)

In [ ]:
global_data = pd.merge(dato, result, on=['country', 'Year'], how='left')
global_data

In [ ]:
global_data = global_data.drop_duplicates(subset=['Trade.Agreement', 'country'])
global_data

In [ ]:
global_data = global_data.rename(columns = {'Annual CO2 emissions (t)': 'co2e'})
global_data = global_data.rename(columns = {'Foreign direct investment, net inflows (% of GDP)': 'ide'})

In [ ]:
global_data = pd.merge(global_data, EPI20, left_on=['country'], right_on=['C'], how='left')
global_data.drop(columns=['C'], inplace=True)
global_data = pd.merge(global_data, DEM22, left_on=['country'], right_on=['C'], how='left')
global_data.drop(columns=['C'], inplace=True)
global_data = pd.merge(global_data, VCC, left_on=['country'], right_on=['C'], how='left')
global_data.drop(columns=['C'], inplace=True)
global_data = global_data.rename(columns = {'Democracy score': 'democ'})
global_data = global_data.rename(columns = {'Negotiations.of.environmental.agreements': 'nea'})
global_data = global_data.rename(columns = {'PVCCI': 'vcc'})
global_data

##### aggregation

In [ ]:
m1 = global_data.groupby('Trade.Agreement')['Dyads'].max().reset_index()
m2 = global_data.groupby('Trade.Agreement')['NorthSouth'].max().reset_index()
m3 = global_data.groupby('Trade.Agreement')['NumberMS'].max().reset_index()
m4 = global_data.groupby('Trade.Agreement')['Sectoral'].max().reset_index()
m5 = global_data.groupby('Trade.Agreement')['TypeDepth'].max().reset_index()
m6 = global_data.groupby('Trade.Agreement')['TypeMemb'].max().reset_index()
m7 = global_data.groupby('Trade.Agreement')['WTOListed'].max().reset_index()
m8 = global_data.groupby('Trade.Agreement')['DateYMD'].max().reset_index()
m9 = global_data.groupby('Trade.Agreement')['Year'].max().reset_index()
m10 = global_data.groupby('Trade.Agreement')['reg_africa'].max().reset_index()
m11 = global_data.groupby('Trade.Agreement')['reg_americas'].max().reset_index()
m12 = global_data.groupby('Trade.Agreement')['reg_asia'].max().reset_index()
m13 = global_data.groupby('Trade.Agreement')['reg_europe'].max().reset_index()
m14 = global_data.groupby('Trade.Agreement')['reg_intercontinental'].max().reset_index()
m15 = global_data.groupby('Trade.Agreement')['reg_oceania'].max().reset_index()
m20 = global_data.groupby('Trade.Agreement')['nea'].max().reset_index()

In [ ]:
import numpy as np
def custom_agg1(values):
    if values.isnull().any():  # Vérifie si l'une des valeurs agrégées est NA
        return np.nan  # Si c'est le cas, retourne NA
    else:
        return values.mean()  # Sinon, retourne la moyenne des valeurs

def custom_agg2(values):
    if values.isnull().any():  # Vérifie si l'une des valeurs agrégées est NA
        return np.nan  # Si c'est le cas, retourne NA
    else:
        return values.sum()  # Sinon, retourne la moyenne des valeurs

def custom_agg3(values):
    if values.isnull().any():  # Vérifie si l'une des valeurs agrégées est NA
        return np.nan  # Si c'est le cas, retourne NA
    else:
        return values.max()  # Sinon, retourne le max des valeurs

m21 = global_data.groupby('Trade.Agreement')['pop'].agg(custom_agg1).reset_index()
m28 = global_data.groupby('Trade.Agreement')['epi2020'].agg(custom_agg1).reset_index()
m23 = global_data.groupby('Trade.Agreement')['co2e'].agg(custom_agg2).reset_index()

m22 = global_data.groupby('Trade.Agreement')['ide'].agg(custom_agg3).reset_index()
m17 = global_data.groupby('Trade.Agreement')['democ'].agg(custom_agg3).reset_index()
m18 = global_data.groupby('Trade.Agreement')['vcc'].agg(custom_agg1).reset_index()

m24 = global_data.groupby('Trade.Agreement')['gdp'].agg(custom_agg1).reset_index()
m25 = global_data.groupby('Trade.Agreement')['gdp_pcap'].agg(custom_agg1).reset_index()

m26 = global_data.groupby('Trade.Agreement')['export'].agg(custom_agg1).reset_index()
m27 = global_data.groupby('Trade.Agreement')['merch'].agg(custom_agg1).reset_index()

In [ ]:
global_data_df = pd.concat([m1, m2['NorthSouth'], m3['NumberMS'], m4['Sectoral'], m5['TypeDepth'], m6['TypeMemb'], m7['WTOListed'],
                       m8['DateYMD'], m9['Year'], m20['nea'], m21['pop'], m24['gdp'], m25['gdp_pcap'], m26['export'], m27['merch'],
                       m23['co2e'], m28['epi2020'], m22['ide'], m17['democ'], m18['vcc']], axis=1)
global_data_df

In [ ]:
cut = pd.to_datetime('1990-01-01')
global_fil = global_data_df[global_data_df['DateYMD'] >= cut]
global_fil

In [ ]:
coup = df_scores.drop(df_scores.columns[1:19], axis = 1)
an_trend_90s = pd.merge(global_fil, coup, on = ['Trade.Agreement'], how = 'left')
an_trend_90s

In [ ]:
break

In [ ]:
# an_trend_90s.to_csv('an_trend_eco90s.csv', index = False)

In [ ]:
an_trend_90s.isna().sum()

**Enfin j'ai finis avec ces données :/**